In [ ]:
import torch
import dfencoder
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
from dfencoder import AutoEncoder
torch.cuda.empty_cache()

In [ ]:
train_path = "C:/Users/Arjun/kaggle/porto-seguro/train.csv"
test_path = "C:/Users/Arjun/kaggle/porto-seguro/test.csv"
submission_path = "C:/Users/Arjun/kaggle/porto-seguro/sample_submission.csv"

In [ ]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
to_remove = ['id']
binary = []
cats = []

for col in train.columns:
    if 'calc' in col:
        to_remove.append(col)
    elif 'bin' in col:
        binary.append(col)
    elif 'cat' in col:
        cats.append(col)

In [ ]:
#drop undesired columns.
X_train = train.drop(columns=to_remove+['target'])
Y_train = train['target']

In [ ]:

X_test = test.drop(columns=to_remove)

In [ ]:
#do our one-hot encoding, naming new columns.
for cat in cats:
    ohe = pd.get_dummies(X_test[cat])
    ohe_columns = [cat + "_ohe_" + str(i) for i in ohe.columns]
    ohe.columns = ohe_columns
    X_test = pd.concat([X_test, ohe], axis=1)

    ohe = pd.get_dummies(X_train[cat])
    ohe.columns = ohe_columns
    X_train = pd.concat([X_train, ohe], axis=1)

    binary += ohe_columns

In [ ]:

#set all column types to numeric
for col in X_train.columns:
    if col in binary:
        X_train[col] = X_train[col].astype(int)
        X_test[col] = X_test[col].astype(int)

In [ ]:

#our dataframe should have 221 columns, all numeric.
%time
X_train.head()

In [ ]:

#validation sample
X_val = X_test.sample(frac=.01)

In [ ]:
#save the encoded test set for later
X_test.to_hdf('X_test.h5', key='stage', mode='w')



#delete references to memory-consuming data
import gc

del train
del test
del X_test

#remove objects from memory
gc.collect()
torch.cuda.empty_cache()

### When to use torch.cuda.empty_cache()?

One case where you would want to use it though is if you use cudnn benchmark mode (torch.backends.cudnn.benchmark=True), then you can add one after the very first forward of the program that you do. As the benchmark mode can allocate large memory blocks during the very first forward to test algorithms and that won’t be good latter.


In [ ]:

#dfencoder allows us to specify scaling behavior per-feature;
#this dictionary will specify not to scale one-hot encoded features.
scaler_spec = {ft:None for ft in binary}

#### Validation on Hold-out set

This is the unsupervised learning. EVen here it is good idea to split train/val set during training. This will help to fune tune the model and avoid overfitting. Holdout : 20%

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .2, random_state =42) 

In [ ]:
encoder = AutoEncoder(
#imput and output layers are implicit
    encoder_layers = [1500, 1500, 1500],
    decoder_layers = [],
    lr = 2.8e-3,
    batch_size = 128,
    activation = 'relu',
    lr_decay =.995,
    swap_p = 0.2,
    #swap noise
    # "scalar_spec" do not scale following cols
    #when you put gauss_rank, you are scaling the specific columns. 
    scaler = 'gauss_rank',
    optimizer ='adam',
    verbose = False,
    logger = 'ipynb',
    progress_bar = True,
    device = torch.device("cuda:0")   
)

In [ ]:
encoder.fit(X_train, val=X_val, epochs=500)


In [ ]:
z = encoder.get_deep_stack_features(X_val)
print(z.shape)
print(z[0, :])
del z
torch.cuda.empty_cache()


In [ ]:
class ClassifierModel ((torch.nn.Module)):
    def __Init__ ( self, *args, **kwargs):
        # we are calling __init__ of parent class. i.e torch.nn.module
        super(ClassifierModel, self).__init__(*args, **kwargs)
        self.input_dropout = torch.nn.Dropout(.1)
        self.input_layer = torch.nn.Linear(4500.1000)
        self.dropout = troch.nn.Dropout(.5)
        self.dense1 = torch.nn.Linear(1000,10000)
        # in keras you set one value for dense egL Dense(128, activation ='relu)
        #alternatively in pytorch you set torch.nn.Linear(128, output_value)
        self.dense2 = torch.nn.Linear(1000,1000)
        self.relu = torch.nn.ReLU()
        self.output = torch.nn.Linear(1000,1)
        self.sigmoid = torch.nn.Sigmoid
        
        def forward(self, x):
            x = self.input_dropout(x)
            
            #layer1
            x = self.dropout(self.relu(self.input_layer(x)))
           
            #layer 2
            x = self.dropout(self.relu(self.dense1(x)))
            
            #layer 3
            x  = self.dropout(self.relu(self.dense2(x)))
            
            #output layer
            x = self.sigmoid(self.output(x))
            return x
        
#create classifier object

classifier = ClassifierModel()

In [ ]:
optim = torch.optim.SGD(
    classifier.parameters(),
    lr=1e-4,
    weight_decay=.05,
    
)

decay = torch.optim.lr_scheduler.ExponentialLR(optim, .995)

loss = torch.nn.modules.loss.BCELoss()

In [ ]:
classifier = classifier.to("cuda:0")

In [ ]:
gc.collect()

In [ ]:
 torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm_notebook as tqdm

from sklearn.metrics import f1_score
import numpy as np
import torch

import tqdm
from IPython.display import clear_output

def do_step(classifier, optim, z, target, loss):
    pred = classifier(z)
    target = torch.tensor(target).float().reshape(-1, 1).to("cuda:0")
    loss_ = loss(pred, target)
    amnt = loss_.item()
    loss_.backward()
    optim.step()
    optim.zero_grad()
    return amnt

def do_evaluation(classifier, z, target, loss):
    with torch.no_grad():
        pred = classifier(z)
        target_ = torch.tensor(target).float().reshape(-1, 1).to("cuda:0")
        loss_ = loss(pred, target_)
        return loss_.item()
  
batch_size =128
n_updates = (len(X_train)//batch_size) + 1

n_megabatches = 8
n_epochs = 150

n_rows = len(X_train)

res = n_rows/n_megabatches #4651/8
batches_per_megabatch = (res // batch_size) + 1 # 581 /128 =4 (floor division)
megabatch_size = batches_per_megabatch * batch_size # 4*128 =512
final_batch_size = n_rows - (n_megabatches - 1) * megabatch_size #4651 - 7 = 4644

for epoch in range(n_epochs):
    with tqdm.tqdm(total=n_updates) as bar:
        for j in range(n_megabatches):
            mega_start = int((j) * megabatch_size) #512
            mega_stop = int((j+1) * megabatch_size) #1024
            slc = X_train.iloc[mega_start:mega_stop] #from 512 to 1024
            z = encoder.get_deep_stack_features(slc) #get z from 512 to 1024
            target_slc = Y_train.iloc[mega_start:mega_stop].values
            if j == (n_megabatches-1):
                steps = final_batch_size/batch_size
            #if final_batch_size % batch_size != 0:
                #steps = steps +1
            else:
                steps = int(batches_per_megabatch)
            steps = int(steps)
            for i in range(steps):
                step = i
                start = int((step) * batch_size)
                stop = int((step+1) * batch_size)
                in_ = z[start:stop]
                target = target_slc[start:stop]
                do_step(classifier, optim, in_, target, loss)
                bar.update(1)
                bce_loss = do_evaluation(classifier, in_, target, loss)
            decay.step()
            
    del z
    torch.cuda.empty_cache()

print('validation loss: ', round(bce_loss, 4))


In [ ]:
X_test = pd.read_hdf('X_test.h5')
test = pd.read_csv(test_path)
ids = test['id'].values


In [ ]:
ids.shape

In [ ]:
del test

In [ ]:
# we are moving the tensor to GPU
def do_inference(encoder, classifier, data):
    #z = torch.tensor(encoder.get_deep_stack_features(data)).to("cuda:0")
    #or you can do torch.tensor(encoder.get_deep_stack_features(data)).cuda()
    z = torch.tensor(encoder.get_deep_stack_features(data)).cuda()
    output = classifier(z)
    return output

In [ ]:
eval_batch_size = 2048
n_batches = (len(X_test)//eval_batch_size) + 1
#n_batches = 10

out = []

with torch.no_grad():
    for i in tqdm.trange(n_batches):
        start = i * eval_batch_size
        stop = (i+1) * eval_batch_size
        slc = X_test.iloc[start:stop]
        res = do_inference(encoder, classifier, slc)
        out.append(res.reshape(-1))
        clear_output()

In [ ]:

import numpy as np

# we are taking vec tensor to CPU using cpu()

result = np.concatenate([vec.cpu().numpy() for vec in out])

In [ ]:
result.shape

#### Format data for kaggle submission

In [ ]:
to_submit = pd.DataFrame()
to_submit['id'] = ids
to_submit['target'] = result

to_submit.to_csv('submission.csv', index=False, encoding='utf-8')